In [1]:
# Apply model on test set
# Already got subset with bcftools, extracted SNPs and dosage

In [1]:
import sqlite3
import argparse
import datetime
import time
import numpy as np
import os

In [29]:
print('Last run:', datetime.datetime.now().strftime('%Y-%m-%d'))
start_time = time.time()

parser = argparse.ArgumentParser()
parser.add_argument('--dosage', default='/data100t1/home/wanying/CCHC/lipidomics/prediction_models/input_docs/sqlDB/dosage_test.db',
                    help='Input SQL database containing SNP dosage')
parser.add_argument('--coeff', default='/data100t1/home/wanying/CCHC/lipidomics/prediction_models/elastic_net/training/model_params/coeff_100_alpha.db',
                    help='Coefficient SQL database file')
parser.add_argument('--output', default='/data100t1/home/wanying/CCHC/lipidomics/prediction_models/elastic_net/test/pred_vals.txt',
                    help='Output filename (including directory)')
parser.add_argument('--lipid_list', default='/data100t1/home/wanying/CCHC/lipidomics/prediction_models/input_docs/lipid_traits_residuals/train/lipid_species_list.txt',
                    help='A list of lipid to test. One lipid per row without header line.')
args = parser.parse_args('--dosage /data100t1/home/wanying/CCHC/lipidomics/prediction_models/input_docs/sqlDB/dosage_test.db'.split())

# if os.path.isfile(args.output):
#     print(f'#Output file already exists: {args.output}')
#     print('#Skip saving. Exit')
#     exit()

# Connect to databases
print('#Connect to dosage and coefficient databases')
con_coeff = sqlite3.connect(args.coeff)
cur_coeff = con_coeff.cursor()

con_dosage = sqlite3.connect(args.dosage)
cur_dosage = con_dosage.cursor()
# Get sample IDs
headers = cur_dosage.execute('SELECT * FROM dosage').description
sample_ids = [val[0] for val in headers][6:]

print('#Load lipid list')
fh_lipid_list = open(args.lipid_list)
lipid = fh_lipid_list.readline().strip()
count = 1
# while lipid != '':
#     print(f'# - Process lipid #{count}: {lipid}')

#     pred_vals = get_pred_vals(lipid, cur_coeff, cur_dosage, sample_ids)
#     lipid = fh_lipid_list.readline().strip()
#     count += 1

res_coeff = cur_coeff.execute(f"SELECT snp_id, weight FROM weights WHERE lipid='{lipid}'")
result_coeff = res_coeff.fetchall()
snp = ["'"+val[0]+"'" for val in result_coeff] # Load SNP IDs
coeff_array = np.array([val[1] for val in result_coeff]) # Load coefficients

# Order is retained as original order in the database. So SNPs and coefficients are lined up already, no need to sort
res_dosage = cur_dosage.execute(f"SELECT {','.join(sample_ids)} FROM dosage WHERE ID IN ({','.join(snp)})")
dosage = res_dosage.fetchall()

end_time = time.time()
print(f'#Run finished in {(end_time-start_time)/60:.4f} min')


# fh_lipid_list.close()
# con_coeff.close()
# con_dosage.close()

Last run: 2023-05-01
#Connect to dosage and coefficient databases
#Load lipid list
#Run finished in 0.1257 min


In [39]:
print(len(dosage), len(dosage[0]))

1251 682


In [32]:
dosage_array = np.array(dosage)

In [33]:
dosage_array.shape

(1251, 682)

In [34]:
dosage_array

array([[0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
       [0.   , 0.128, 1.997, ..., 0.951, 0.951, 0.   ],
       ...,
       [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , ..., 0.149, 0.149, 0.   ]])

In [36]:
print(coeff_array.shape)
coeff_array

(1255,)


array([-0.01922263, -0.01071893, -0.00084994, ...,  0.02382914,
        0.03180891,  0.00399275])

In [45]:
[x[0] for x in cur_dosage.execute(f"SELECT * FROM dosage").description[:10]]

['CHROM',
 'POS',
 'ID',
 'REF',
 'ALT',
 'INFO',
 'BD0010_BD4010',
 'BD0382_BD4382',
 'BD0550_BD4550',
 'BD0859_BD4859']

In [50]:
missing = []
for s in snp:
    number_find = len(cur_dosage.execute(f"SELECT ID FROM dosage WHERE ID={s}").fetchall())
    if number_find==0:
        missing.append(f'{s}|{number_find}')
    else: print('.', end='', flush=True)
print('################# Missing #################')
for val in missing:
    print(val)

.........................................................................................................................................................................................

KeyboardInterrupt: 

In [51]:
missing

["'chr1:10698556:G:GA'|n0"]

In [52]:
tmp = cur_dosage.execute(f"SELECT ID FROM dosage WHERE CHROM=1 AND POS=10698556").fetchall()

In [53]:
len(tmp)

1

In [55]:
tmp

[('chr1:10698556:G:A',)]